In [1]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing   import LabelEncoder, OneHotEncoder
from imblearn.over_sampling  import SMOTE
from sklearn.preprocessing   import StandardScaler
from sklearn.metrics         import f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV 
from lightgbm                import LGBMClassifier

In [2]:
df = pd.read_csv('train.csv')

In [3]:
x = df.iloc[:, 1:-1]
y = df.iloc[:,-1]

In [4]:
df.columns

Index(['loan_id', 'source', 'financial_institution', 'interest_rate',
       'unpaid_principal_bal', 'loan_term', 'origination_date',
       'first_payment_date', 'loan_to_value', 'number_of_borrowers',
       'debt_to_income_ratio', 'borrower_credit_score', 'loan_purpose',
       'insurance_percent', 'co-borrower_credit_score', 'insurance_type', 'm1',
       'm2', 'm3', 'm4', 'm5', 'm6', 'm7', 'm8', 'm9', 'm10', 'm11', 'm12',
       'm13'],
      dtype='object')

In [5]:
x = x.drop(labels = ['source','financial_institution','loan_purpose','first_payment_date','origination_date'], axis = 1)
x['avg_score'] = (x['borrower_credit_score']  + x['co-borrower_credit_score'] )/x['number_of_borrowers']
x['loan_term'] = x['loan_term']/30
x_m14 = df.iloc[:, 16:-1]
x['m14'] = x_m14.sum(axis = 1)

In [6]:
for item in x.columns:
    x[item + '_log']   = np.log(x[item])
    x[item +'_power'] = np.power(x[item],2)
    x[item + '_power_log'] = np.log(x[item+'_power'])
    x[item + '_log_power'] = np.power(x[item+'_log'],2)

C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [7]:
x = np.nan_to_num(x)
sc = StandardScaler()
x = sc.fit_transform(x)
x = np.nan_to_num(x)

C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:771: RuntimeWarning: invalid value encountered in true_divide
  X /= se

In [8]:
y = y.values

In [9]:
skf = StratifiedKFold(n_splits=5)
smote = SMOTE(sampling_strategy='minority', ratio= 0.35,k_neighbors=7)

In [10]:
%%time
for train_index, test_index in skf.split(x, y):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    x_train, y_train = smote.fit_sample(x_train, y_train)

    
    classifier_lgbm = LGBMClassifier(boosting_type= 'gbdt', application = 'binary', num_iterations = 100, learning_rate = 0.005, num_leaves = 8,
          objective = 'binary', device = 'cpu', max_depth = -1, max_bin = 255, lambda_l1 = 5, lambda_l2 = 10, 
          metric = 'binary_error', subsample_for_bin = 200, subsample  = 0.7, colsample_bytree = 0.64, min_child_weight = 1,
          min_child_samples = 5, reg_alpha = 1.2, reg_lambda = 1, n_jobs = 5, silent = True )
    classifier_lgbm.fit(x_train, y_train)
    y_pred_lgbm = classifier_lgbm.predict(x_test)

    
    print (f1_score(y_test,y_pred_lgbm))
    print (confusion_matrix(y_test,y_pred_lgbm))
    



C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.5381526104417671
[[23031    54]
 [   61    67]]


C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.48717948717948717
[[23035    50]
 [   70    57]]


C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.5462555066079294
[[23046    38]
 [   65    62]]


C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.5955555555555555
[[23053    31]
 [   60    67]]


C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.48034934497816595
[[23037    47]
 [   72    55]]
Wall time: 7.46 s


# Generate the Test file here for submission 

In [431]:
df_test  = pd.read_csv('test.csv')
final_df = pd.DataFrame({"loan_id": df_test['loan_id']})

In [432]:
df_test = df_test.drop(labels = ['loan_id','source','financial_institution','loan_purpose','first_payment_date','origination_date',], axis = 1)
df_test['avg_score'] = (df_test['borrower_credit_score']  + df_test['co-borrower_credit_score'] )/df_test['number_of_borrowers']
df_test['loan_term'] = df_test['loan_term']/30
df_m14 = df_test.iloc[:, 16:-1]
df_test['m14'] = df_m14.sum(axis = 1)

In [434]:
for item in df_test.columns:
    df_test[item + '_log']       = np.log(df_test[item])
    df_test[item +'_power']      = np.power(df_test[item],2)
    df_test[item + '_power_log'] = np.log(df_test[item+'_power'])
    df_test[item + '_log_power'] = np.power(df_test[item+'_log'],2)

C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [435]:
df_test = np.nan_to_num(df_test)
sc = StandardScaler()
df_test = sc.fit_transform(df_test)
df_test = np.nan_to_num(df_test)

C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\Users\ku71tcd\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\data.py:771: RuntimeWarning: invalid value encountered in true_divide
  X /= se

In [436]:
df_test_pred    = classifier_lgbm.predict(df_test)

In [437]:
final_df["m13"] = df_test_pred
final_df.to_csv("creditrisk_lgbm.csv",index=False)